<a href="https://colab.research.google.com/github/AnastasiiaPashkevych/database-querying-using-SQLite/blob/main/Database_Querying_using_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [2]:
!pip install ipython-sql
%load_ext sql

import pandas as pd
import csv, sqlite3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.2 MB/s eta 0:00:00


In [5]:
# Establish connection to SQLite database
conn = sqlite3.connect("FinalDB.db")

# Create a cursor object
cursor = conn.cursor()

# Print confirmation message
print("Connected to FinalDB.db successfully")


Connected to FinalDB.db successfully


In [6]:
# Load datasets into pandas DataFrames
census_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv"
schools_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv"
crime_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv"

census_df = pd.read_csv(census_url)
schools_df = pd.read_csv(schools_url)
crime_df = pd.read_csv(crime_url)

# Load data into the database
census_df.to_sql("CENSUS_DATA", conn, if_exists="replace", index=False)
schools_df.to_sql("CHICAGO_PUBLIC_SCHOOLS", conn, if_exists="replace", index=False)
crime_df.to_sql("CHICAGO_CRIME_DATA", conn, if_exists="replace", index=False)

533

In [7]:
# Connect SQL magic module to the database
%sql sqlite:///FinalDB.db

In [8]:
# Find the total number of crimes recorded in the CRIME table.

%%sql
SELECT COUNT(*) AS total_crimes FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


total_crimes
533


In [9]:
# List community area names and numbers with per capita income less than 11000.

%%sql
SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER, PER_CAPITA_INCOME
FROM CENSUS_DATA
WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,PER_CAPITA_INCOME
West Garfield Park,26.0,10934
South Lawndale,30.0,10402
Fuller Park,37.0,10432
Riverdale,54.0,8201


In [11]:
# List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

%%sql
SELECT CASE_NUMBER
FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%MINOR%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


In [8]:
# List all kidnapping crimes involving a child?

%%sql
SELECT *
FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%';

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


In [9]:
# List the kind of crimes that were recorded at schools. (No repetitions)

%%sql
SELECT DISTINCT PRIMARY_TYPE
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [11]:
# List the type of schools along with the average safety score for each type.

%%sql
SELECT "Elementary, Middle, or High School" AS School_Type, AVG(SAFETY_SCORE) AS Average_Safety_Score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


School_Type,Average_Safety_Score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [12]:
# List 5 community areas with highest % of households below poverty line

%%sql
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [13]:
# Which community area is most crime prone? Display the coumminty area number only.

%%sql
SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS Crime_Count
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY Crime_Count DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,Crime_Count
25.0,43


In [ ]:
# Use a sub-query to find the name of the community area with highest hardship index

%%sql
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX)
    FROM CENSUS_DATA
);

In [15]:
# Use a sub-query to determine the Community Area Name with most number of crimes?

%%sql
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin


In [ ]:
conn.close()